In [4]:
pip install razdel

In [5]:
pip install pymorphy2[fast]

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 8.2MB 7.1MB/s 
     |████████████████████████████████| 378kB 38.4MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=869158 sha256=0c2b2b5d93f98a17fef88ac0adc8f9b81458786415d9585ee8394abfc3ee64eb
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [1]:
pip install nltk

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
pip install np

  Created wheel for np: filename=np-1.0.2-cp37-none-any.whl size=13652 sha256=37ac68d05222b772bf4ea14d62b3e88b1d27855169c071558c3494abe0e0bad4
  Stored in directory: /root/.cache/pip/wheels/2b/df/57/f40bef951382112d9c644b3ec6d713ceb200ea7c15c074de42
Successfully built np


In [4]:
pip install numpy

In [22]:
import itertools
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
from nltk.corpus import stopwords

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [7]:
from google.colab import files 
uploaded = files.upload()

Saving lenta.txt to lenta (1).txt


In [8]:
c2 = open('lenta.txt')

In [9]:
c3 = c2.readlines()

In [15]:
corpus = open('lenta.txt').read()[:300000]

In [10]:
from collections import Counter

In [11]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [12]:
from nltk.tokenize import sent_tokenize

In [13]:
import numpy as np

In [13]:
nltk.download('punkt')

NameError: ignored

In [16]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(corpus)]

In [17]:
import nltk
from nltk.collocations import *

In [18]:
unigrams_news = Counter()
bigrams_news = Counter()
trgr_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n = 2))
    trgr_news.update(ngrammer(sentence))

In [ ]:
bigrams_news.most_common(50)

In [2]:
unigrams_news.most_common(50)

NameError: ignored

In [19]:

matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
bigrams_id2word_news = list(bigrams_news)
bigrams_word2id_news = {word:i for i, word in enumerate(bigrams_id2word_news)}
# вероятность расчитываем точно также
for ngram in trgr_news:
    slot1 = ' '.join(ngram.split()[:2])
    word3 = ngram.split()[2]
    matrix_news[bigrams_word2id_news[slot1]][word2id_news[word3]] =  (trgr_news[ngram]/
                                                                     bigrams_news[slot1])

In [20]:
def generate(matrix, id2word, bigrams_word2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigrams_word2id[start]
    current_bigram = start

    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        current_bigram = current_bigram.split()[1] + ' ' + id2word[chosen]

        if id2word[chosen] == '<end>':
            chosen = bigrams_word2id['<start> <start>']
            current_bigram = '<start> <start>'
        current_idx = bigrams_word2id[current_bigram]
    return ' '.join(text)

In [21]:
print(generate(matrix_news, id2word_news, bigrams_word2id_news).replace('<end>', '\n'))

пока полиция добиралась до места падения 
 в 1.30 мск напомощь наступающим вновь пришла авиация 
 касаясь деловых контактов с фирмой mabetex 1 миллиона американских долларовна счет банка в будапеште в пользу бориса ельцина дляоплаты его расходов во время воздушных разведок русских авиаторов австрийцы всегда безошибочно определяли каким аппаратом управлял нестеров 
 здесь задолженность по зарплате и другим спецслужбам сша ведущим расследование 
 в настоящее время анализируется 
 по его мнению она способна передать в обмен на гарантии всей группы в узбекистан 
 однако осман оджалан обещает что вывод войск завершится за пять дней однако пока курды вывели только четверть


2 задача

In [23]:
corpus = preprocess(corpus)

In [24]:
def pmi(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    try:
      pmi = np.log((bigram_count/corpus_word_count) / ((worda_count/corpus_word_count) * (wordb_count/corpus_word_count)))
    except ZeroDivisionError:
      return 0
      
    return pmi

In [26]:
import gensim

In [28]:
ph = gensim.models.Phrases(corpus, scoring=pmi, threshold=0.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[100]]]

['говориться_поступить',
 'сегодня_сообщение',
 'центр_общественный',
 'связь_фсб',
 'соответствие_основа',
 'концепция_государственный',
 'политика_российский',
 'федерация_военномустроительство',
 'период_2005',
 'год_база',
 'департамент_борьба',
 'терроризм_управление',
 'конституционный_безопасность',
 'фсб_создать',
 'департамент_защита',
 'конституционный_строй',
 'борьба_терроризм']

In [29]:
ph = gensim.models.Phrases(corpus, scoring=pmi, threshold=30.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[100]]]

['говориться',
 'поступить',
 'сегодня',
 'сообщение',
 'центр',
 'общественный',
 'связь',
 'фсб',
 'соответствие',
 'основа',
 'концепция',
 'государственный',
 'политика',
 'российский_федерация',
 'военномустроительство',
 'период',
 '2005',
 'год',
 'база',
 'департамент',
 'борьба_терроризм',
 'управление',
 'конституционный',
 'безопасность',
 'фсб',
 'создать',
 'департамент',
 'защита',
 'конституционный',
 'строй',
 'борьба_терроризм']

In [30]:
ph = gensim.models.Phrases(corpus, scoring=pmi, threshold=100.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[100]]]

['говориться',
 'поступить',
 'сегодня',
 'сообщение',
 'центр',
 'общественный',
 'связь',
 'фсб',
 'соответствие',
 'основа',
 'концепция',
 'государственный',
 'политика',
 'российский_федерация',
 'военномустроительство',
 'период',
 '2005',
 'год',
 'база',
 'департамент',
 'борьба_терроризм',
 'управление',
 'конституционный',
 'безопасность',
 'фсб',
 'создать',
 'департамент',
 'защита',
 'конституционный',
 'строй',
 'борьба_терроризм']

In [31]:
ph = gensim.models.Phrases(corpus, scoring=pmi, threshold=0.05)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[100]]]

['говориться_поступить',
 'сегодня_сообщение',
 'центр_общественный',
 'связь_фсб',
 'соответствие_основа',
 'концепция_государственный',
 'политика_российский',
 'федерация_военномустроительство',
 'период_2005',
 'год_база',
 'департамент_борьба',
 'терроризм_управление',
 'конституционный_безопасность',
 'фсб_создать',
 'департамент_защита',
 'конституционный_строй',
 'борьба_терроризм']